# Big Data Modelling and Management - Database Design

**Group 24 members:** <br>
Filipe Lourenço (R20170799), <br>
Guilherme Neves (R20170749), <br>
Rui Monteiro (R20170796), <br>
Vasco Pestana (R20170803)

**MSc:** Data Science and Advanced Analytics - Nova IMS <br>
2020/2021

In [2]:
from pymongo import MongoClient, DESCENDING, ASCENDING, TEXT, GEO2D
import mysql.connector
from bson.objectid import ObjectId
from bson.decimal128 import Decimal128
import datetime
from decimal import *
from urllib.request import urlopen
import json
import csv
from copy import deepcopy
import pandas as pd
from math import isnan

### Connecting to the MongoDB database

In [3]:
host="rhea.isegi.unl.pt"
port="27041"
user="GROUP_24"
password="T0b5Sez8prB8jo9ohdb907GL5U0S24GX"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WWI

## 1. Migrate Data to MongoDB
### 1.1. Migrate Data from the SQL Tables

In [194]:
mydb = mysql.connector.connect(host='rhea.isegi.unl.pt',
                               user='wwi-read-only-user',
                               database='WWI',
                               port=3306,
                               password='jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb')
mycursor = mydb.cursor()

mycursor.execute('SHOW TABLES;')
print(f"Tables:\n {mycursor.fetchall()}\n")

Tables:
 [('Purchasing_PurchaseOrderLines',), ('Purchasing_PurchaseOrders',), ('Purchasing_SupplierCategories',), ('Purchasing_SupplierTransactions',), ('Purchasing_Suppliers',), ('Sales_CustomerCategories',), ('Sales_CustomerTransactions',), ('Sales_Customers',), ('Sales_InvoiceLines',), ('Sales_Invoices',), ('Sales_OrderLines',), ('Sales_Orders',)]



In [195]:
mycursor.execute('DESCRIBE Sales_CustomerTransactions;')
print(f"Tables:\n {mycursor.fetchall()}\n")

Tables:
 [('CustomerTransactionID', b'int', 'YES', '', None, ''), ('CustomerID', b'int', 'YES', '', None, ''), ('TransactionTypeID', b'int', 'YES', '', None, ''), ('InvoiceID', b'int', 'YES', '', None, ''), ('PaymentMethodID', b'int', 'YES', '', None, ''), ('TransactionDate', b'date', 'YES', '', None, ''), ('AmountExcludingTax', b'decimal(18,2)', 'YES', '', None, ''), ('TaxAmount', b'decimal(18,2)', 'YES', '', None, ''), ('TransactionAmount', b'decimal(18,2)', 'YES', '', None, ''), ('OutstandingBalance', b'decimal(18,2)', 'YES', '', None, ''), ('FinalizationDate', b'date', 'YES', '', None, ''), ('IsFinalized', b'bit(1)', 'YES', '', None, '')]



In [9]:
def insert_sql(collection_name):
    # Connect to the cursor
    mydb = mysql.connector.connect(host='rhea.isegi.unl.pt',
                               user='wwi-read-only-user',
                               database='WWI',
                               port=3306,
                               password='jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb')
    mycursor = mydb.cursor()

    # Get tables' columns
    mycursor.execute('DESCRIBE ' + collection_name + ';')
    table_columns = mycursor.fetchall()
    
    field_names = []
    for i in range(len(table_columns)):
        field_names.append(table_columns[i][0])
    
    mycursor.execute(' SELECT ' + ', '.join(field_names) + ' FROM ' + collection_name)

    # Create the new collection
    db.create_collection(collection_name)

    # pymongo requires Python dictionaries
    cus = dict()

    # Cycle through each MySQL row in the cursor
    for row in mycursor:
        # Insert an ObjectId to the document
        cus['_id'] = ObjectId()                                   

        # Check if the current row is null
        for i in range(0, len(row)):
            if (row[i] == None) or (row[i] == ''):
                # If the record is null OR it is an empty string, skip it
                continue
            else:
                value=row[i]
                # If the record is in a data type that pymongo doesn't accept, switch it to the right data type
                if type(value) == Decimal:
                    value = Decimal128(value)
                elif type(value) == datetime.date:
                    value = datetime.datetime(value.year, value.month, value.day)
                    
                # Conversion to string
                row_title = "".join( field_names[i] )  

                # Add current record's field's title and value             
                cus[row_title] = value
                
        # We've completed processing this row, insert it into the MongoDB collection      
        id = db[collection_name].insert_one(cus).inserted_id

Insert the tables stored in the SQL database into the collections of the MongoDB database using the previous function:

In [10]:
# insert_sql('Purchasing_PurchaseOrderLines')
# insert_sql('Purchasing_PurchaseOrders')
# insert_sql('Purchasing_SupplierCategories')
# insert_sql('Purchasing_Suppliers')
# insert_sql('Sales_CustomerCategories')
# insert_sql('Sales_Customers')
# insert_sql('Sales_InvoiceLines')
# insert_sql('Sales_OrderLines')
# insert_sql('Sales_Orders')
# insert_sql('Purchasing_SupplierTransactions')
# insert_sql('Sales_CustomerTransactions')
# insert_sql('Sales_Invoices')

### 1.2. Migrate Data from the API (JSON files)

In [5]:
# Load all JSON files from the API into Python

Application_DeliveryMethods_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_DeliveryMethods.json")\
                                         .read())

Application_Cities_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_Cities.json")\
                                         .read())

Application_PaymentMethods_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_PaymentMethods.json")\
                                         .read())

Application_Countries_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_Countries.json")\
                                         .read())

Application_People_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_People.json")\
                                         .read())

Application_StateProvinces_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_StateProvinces.json")\
                                         .read())

Application_TransactionTypes_json = json.loads(urlopen("http://rhea.isegi.unl.pt:8080/Project_WWI_Application_TransactionTypes.json")\
                                         .read())

In [6]:
collection_names = ['Application_DeliveryMethods', 'Application_Cities', 'Application_PaymentMethods', 'Application_Countries',
                    'Application_People', 'Application_StateProvinces', 'Application_TransactionTypes']

json_files = [Application_DeliveryMethods_json, Application_Cities_json, Application_PaymentMethods_json, Application_Countries_json, 
              Application_People_json, Application_StateProvinces_json, Application_TransactionTypes_json]

In [11]:
for j in range(len(collection_names)):
    # Create the new collection
    db.create_collection(collection_names[j])
    
    list_ = []

    for doc in json_files[j]:
        # Get a copy of the doc/dict
        temp_doc = deepcopy(doc)

        # Check if the current field is null
        for i in doc.keys():
            if (doc[i] == None) or (doc[i] == ''):
                # If the record is null OR it is an empty string, skip it
                temp_doc.pop(i)
                
            else:
                # If the record is in a data type that pymongo doesn't accept, switch it to the right data type
                if type(doc[i]) == Decimal:
                    temp_doc[i] = Decimal128(doc[i])
                elif type(doc[i]) == datetime.date:
                    temp_doc[i] = datetime.datetime(temp_doc[i].year, temp_doc[i].month, temp_doc[i].day)

        # Insert an ObjectId to the document
        temp_doc['_id'] = ObjectId()

        list_.append(temp_doc)
       
    # Insert all docs in the respective collection
    id = db[collection_names[j]].insert_many(list_)

### 1.3. Migrate Data from the CSV files

In [ ]:
collections_names = ['Warehouse_Colors', 'Warehouse_PackageTypes', 'Warehouse_StockGroups', 'Warehouse_StockItems',
                     'Warehouse_StockItemStockGroups', 'Warehouse_StockItemTransactions']

csv_paths = ['Project_WWI_Warehouse_Colors.csv','Project_WWI_Warehouse_PackageTypes.csv','Project_WWI_Warehouse_StockGroups.csv'
             ,'Project_WWI_Warehouse_StockItems.csv','Project_WWI_Warehouse_StockItemStockGroups.csv', 'Project_WWI_Warehouse_StockItemTransactions.csv']

In [ ]:
for coll in range(len(collections_names)):

    for j in range(len(csv_paths)):
        # If the coll is equal to j in csv paths, it means they refer the same csv
        if coll == j:
            # Create the new collection
            db.create_collection(collections_names[coll])
            reader = pd.read_csv(csv_paths[j]).to_dict(orient='records')
            list_ = []
    
            for doc in reader:
                # Create a copy of the original document to make changes
                temp_doc = deepcopy(doc)
                
                for key in doc.keys():
                    # If the value of the key of the document is a string, skip it
                    if isinstance(doc[i], str): 
                        continue
                    # If the value of the key of the document is a nan, delete the key and the value from that document 
                    elif math.isnan(doc[i]):
                        temp_doc.pop(i)
                
                # Add the ObjectId to uniquely identify a document to the copy of the original document and append it to a list_
                temp_doc['_id'] = ObjectId()
                list_.append(temp_doc)
            
            # Insert all documents of the list_ in the collection created before
            db[collections_names[coll]].insert_many(list_)

## 2. Embedding stage

Embedd Application_StateProvinces and Application_Countries

In [17]:
query = [{
        "$lookup":{
           "from": "Application_Countries",
           "localField": "CountryID",
           "foreignField": "CountryID",
           "as": "country"
        }
    },{
        "$merge" : {
            "into" : "Countries_StateProvinces",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Application_StateProvinces.aggregate(query)

list(r)

[]

Embedd Countries_StateProvinces and Application_Cities

In [19]:
query = [{
        "$lookup":{
           "from": "Countries_StateProvinces",
           "localField": "StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "state"
        }
    },{
        "$merge" : {
            "into" : "Countries_StateProvinces_Cities",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Application_Cities.aggregate(query)

list(r)

[]

Embedd Sales_OrderLines inside Warehouse_StockItems


In [ ]:
query = [{
        "$lookup":{
           "from": "Sales_OrderLines",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "stock_item"
        }
    },{
        "$merge" : {
            "into" : "Sales_OrderLines_Warehouse_StockItems",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Warehouse_StockItems.aggregate(query)

list(r)

Embedd Sales_OrderLines and Sales_Orders

In [4]:
query = [{
        "$lookup":{
           "from": "Sales_OrderLines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "order_lines"
        }
    },{
        "$merge" : {
            "into" : "All_Sales_Orders",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Sales_Orders.aggregate(query)

list(r)

[]

Embedd Application_PaymentMethods inside Purchasing_SupplierTransactions

In [ ]:
query = [{
        "$lookup":{
           "from": "Application_PaymentMethods",
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
           "as": "payment_method"
        }
    },{
        "$merge" : {
            "into" : "Purchasing_SupplierTransactions_PaymentMethod",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Purchasing_SupplierTransactions.aggregate(query)

list(r)

Embedd Application_PaymentMethods inside Sales_CustomerTransactions

In [179]:
query = [{
        "$lookup":{
           "from": "Application_PaymentMethods",
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
           "as": "payment_method"
        }
    },{
        "$merge" : {
            "into" : "Sales_CustomerTransactions_PaymentMethod",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Sales_CustomerTransactions.aggregate(query)

list(r)

Embedd PurchasingSupplierCategories inside PurchasingSuppliers 

In [ ]:
query = [{
        "$lookup":{
           "from": "Purchasing_SupplierCategories",
           "localField": "SupplierCategoryID",
           "foreignField": "SupplierCategoryID",
           "as": "purchasing_suppliers_categories"
        }
    },{
        "$merge" : {
            "into" : "All_Purchasing_Supplier_Categories",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Purchasing_Suppliers.aggregate(query)

list(r)

Embedd All_Purchasing_Supplier_Categories inside Purchasing_SupplierTransactions 

In [6]:
query = [{
        "$lookup":{
           "from": "All_Purchasing_Supplier_Categories",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "purchasing_suppliers_categories_transactions"
        }
    },{
        "$merge" : {
            "into" : "All_Purchasing_Supplier_Categories_Transactions",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Purchasing_SupplierTransactions.aggregate(query)

list(r)

[]

Embedd Sales_InvoiceLines and Sales_Invoices

In [9]:
query = [{
        "$lookup":{
           "from": "Sales_InvoiceLines",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "invoice_lines"
        }
    },{
        "$merge" : {
            "into" : "All_Sales_Invoices",
            "whenMatched": "merge"
        }
    }
]

pipeline = [query]

r = db.Sales_Invoices.aggregate(query)

list(r)

[]

## 3. Create Field

Add the 'Items_Description' array with the Items descriptions for each Order in All_Sales_Orders.

In [4]:
for doc in db.All_Sales_Orders.find():
    list_items = []
    for doc_order in doc['order_lines']:
        list_items.append(doc_order['Description'])
        db.All_Sales_Orders.update_one(doc, { "$set": { "Items_Description": list_items } })

Geospatial field 'DeliveryLocationCoordinates' in Sales_Customers.

In [73]:
for doc in db.Sales_Customers.find():
    list_coords = [] 
    list_coords.append(doc['DeliveryLocationLat'])
    list_coords.append(doc['DeliveryLocationLong'])
    db.Sales_Customers.update_one(doc, { "$set": { "DeliveryLocationCoordinates": list_coords } })

Delete 'DeliveryLocationLat' and 'DeliveryLocationLong' from Sales_Customers.

In [77]:
db.Sales_Customers.update_many({}, {'$unset':{"DeliveryLocationLat":1}})
db.Sales_Customers.update_many({}, {'$unset':{"DeliveryLocationLong":1}})

## 4. Indexes

Ascending index on DeliveryCityID in Purchasing_Suppliers

In [ ]:
db.Purchasing_Suppliers.create_index(
    [('DeliveryCityID', 1)],
    name='DeliveryCityID', 
)

Ascending index on CityID in Countries_StateProvinces_Cities

In [ ]:
db.Countries_StateProvinces_Cities.create_index(
    [('CityID', 1)],
    name='CityID', 
)

Ascending index on DeliveryCityID in Sales_Customers

In [ ]:
db.Sales_Customers.create_index(
    [('DeliveryCityID', 1)],
    name='DeliveryCityID', 
)

Ascending index on OrderID in All_Sales_Orders

In [8]:
db.All_Sales_Orders.create_index(
    [('OrderID', 1)],
    name='OrderID_index', 
)

'OrderID_index'

Ascending index on Description in All_Sales_Orders

In [9]:
db.All_Sales_Orders.create_index(
    [('orderlines.Description', 1)],
    name='Description_index', 
)

'Description_index'

Descending index on Quantity in Sales_OrderLines_Warehouse_StockItems

In [ ]:
db.Sales_OrderLines_Warehouse_StockItems.create_index(
    [('stock_item.Quantity', -1)],
    name='Quantity_index', 
)

Geospatial index in Sales_Customers

In [5]:
db.Sales_Customers.create_index([("DeliveryLocationCoordinates", GEO2D)],
                                  name = 'DeliveryLocationCoordinates_index')

'DeliveryLocationCoordinates_index'

Ascending index on OrderDate in All_Sales_Orders

In [ ]:
db.All_Sales_Orders.create_index(
    [('OrderDate', 1)],
    name='OrderDate_index', 
)

Compound index on TransactionDate and SupplierID in All_Purchasing_Supplier_Categories_Transactions

In [7]:
db.All_Purchasing_Supplier_Categories_Transactions.create_index(
    [('TransactionDate', 1), ('SupplierID', 1)],
    name='TransactionDate_SupplierID_index',
)

'TransactionDate_SupplierID_index'

Ascending index on PaymentMethodID in Purchasing_SupplierTransactions_PaymentMethod

In [ ]:
db.Purchasing_SupplierTransactions_PaymentMethod.create_index(
    [('PaymentMethodID', 1)],
    name='PaymentMethod_index',
)

Ascending index on PaymentMethodID in Sales_CustomerTransactions_PaymentMethod

In [ ]:
db.Sales_CustomerTransactions_PaymentMethod.create_index(
    [('PaymentMethodID', 1)],
    name='PaymentMethod_ID2',
)

Ascending index on PersonID in Application_People

In [ ]:
db.Application_People.create_index(
    [('PersonID', 1)],
    name='PersonID', 
)

Ascending index on SalespersonPersonID in All_Sales_Invoices

In [ ]:
db.All_Sales_Invoices.create_index(
    [('SalespersonPersonID', 1)],
    name='SalespersonPersonID', 
)

Ascending index on FullName in Application_People

In [ ]:
db.Application_People.create_index(
    [('FullName', 1)],
    name='FullName', 
)

Ascending index on InvoiceDate in All_Sales_Invoices

In [ ]:
db.All_Sales_Invoices.create_index(
    [('InvoiceDate', 1)],
    name='InvoiceDate', 
)